In [33]:
import json
import numpy as np 
from pymatgen.command_line.critic2_caller import Critic2Caller
import json
import os 

In [3]:
critic2_input = "../../tests/test_files/critic2/WB2.json"
dict_critic2 = json.load(open(critic2_input, "r"))

In [28]:


target = "/home/santiagovargas/dev/qtaim_generator/tests/test_files/critic2/molecule/phenol_phenol.cri"
os.system("critic2 " + target)

                  _   _     _          ___     
                 (_) | |   (_)        |__ \    
     ___   _ __   _  | |_   _    ___     ) |   
    / __| | '__| | | | __| | |  / __|   / /    
   | (__  | |    | | | |_  | | | (__   / /_    
    \___| |_|    |_|  \__| |_|  \___| |____|   

* CRITIC2: analysis of real-space scalar fields in solids and molecules.
  (c) 1996-2019 A. Otero-de-la-Roza, A. Martin-Pendas, V. Lua~na
  Distributed under GNU GPL v.3 (see COPYING for details)
  Bugs, requests, and rants: aoterodelaroza@gmail.com 
  Website: https://aoterodelaroza.github.io/critic2/
  If you find this software useful, please cite:
  A. Otero-de-la-Roza et al., Comput. Phys. Commun. 185 (2014) 1007-1018.
  A. Otero-de-la-Roza et al., Comput. Phys. Commun. 180 (2009) 157-166.

+ critic2 (development), version 1.1
         host: Linux-6.2.6-76060206-generic
         date: Thu 21 Dec 2023 12:10:53
 compiled dat: /usr/local/share/critic2
      datadir: /usr/local/share/critic2
...was fou

STOP 1


256

In [4]:
from pymatgen.io.xyz import XYZ 
from pymatgen.command_line.critic2_caller import Critic2Analysis
xyz = XYZ.from_file("/home/santiagovargas/dev/qtaim_generator/tests/test_files/critic2/molecule/phenol_phenol.xyz")
mol = xyz.molecule
cro = "../../tests/test_files/critic2/molecule/phenol_phenol.json"
yt = "../../tests/test_files/critic2/molecule/yt.json"
json_out = json.load(open(cro, "r"))

In [102]:

def parse_critic2(cro, features={"atom": [], "bond": []}):
    
    cp_loaded = json.load(open(cro, "r"))
    bohr_to_ang = 0.529177249
    translation = {
        'field' : 'field',
         'gradient_norm' : 'grad_norm',
         'laplacian': "lap_norm",
         'hessian_eigenvalues': 'eig_hess',
         'ellipticity': "ellip_e_dens"
    }
    species = {}
    for specie in cp_loaded["structure"]["species"]:
        if specie["name"][-1] == "_":
            species[specie["id"]] = specie["name"][:-1]
        else:
            species[specie["id"]] = specie["name"]

    atoms = []
    centering_vector = cp_loaded["structure"]["molecule_centering_vector"]

    for ii, atom in enumerate(cp_loaded["structure"]["nonequivalent_atoms"]):
        specie = species[atom["species"]]
        atoms.append(specie)
        tmp = atom["cartesian_coordinates"]
        coords = []
        for jj, val in enumerate(tmp):
            coords.append((val + centering_vector[jj]) * bohr_to_ang)

    if (
        cp_loaded["critical_points"]["number_of_nonequivalent_cps"]
        != cp_loaded["critical_points"]["number_of_cell_cps"]
    ):
        raise ValueError(
            "ERROR: number_of_nonequivalent_cps should always equal number_of_cell_cps!"
        )

    bond_dict = {} # separately parses bonds
    for cp in cp_loaded["critical_points"]["nonequivalent_cps"]:
        if cp["rank"] == 3 and cp["signature"] == -1:
            bond_dict[cp["id"]] = {"field": cp["field"]}
    
    atom_dict = {} # separately parses atoms
    for cp in cp_loaded["critical_points"]["nonequivalent_cps"]:
        if cp["rank"] == 3 and cp["signature"] == -3:
            atom_dict[cp["id"]] = {"field": cp["field"]}

    if features["bond"] != [] or features["atom"] != []:
        cp_features = cp_loaded["critical_points"]["nonequivalent_cps"]
    

    for cp_ind, cp in enumerate(cp_loaded["critical_points"]["cell_cps"]): # finds attractors to BCPs
        if cp["id"] in atom_dict:
            if features["atom"] != []:
                for feature in features["atom"]:
                    if feature in cp_features[cp_ind]:
                        if feature == "ellipticity":
                            atom_dict[cp["id"]]["extra_feat_atom_" + translation[feature]] = np.abs(cp_features[cp_ind]["hessian_eigenvalues"][0]) / np.abs(cp_features[cp_ind]["hessian_eigenvalues"][1]) - 1
                        elif feature == "hessian_eigenvalues":
                            atom_dict[cp["id"]]["extra_feat_atom_" + translation[feature]] = cp_features[cp_ind]["hessian_eigenvalues"][0]           
                        else:
                            atom_dict[cp["id"]]["extra_feat_atom_" + translation[feature]] = cp_features[cp_ind][feature]

    for cp_ind, cp in enumerate(cp_loaded["critical_points"]["cell_cps"]): # finds attractors to BCPs
        if cp["id"] in bond_dict:
            # Check if any bonds include fictitious atoms
            bad_bond = False
            for entry in cp["attractors"]:
                if int(entry["cell_id"]) - 1 >= len(atoms):
                    bad_bond = True
            # If so, remove them from the bond_dict
            if bad_bond:
                bond_dict.pop(cp["id"])

            else:
                bond_dict[cp["id"]]["atom_ids"] = [
                    entry["cell_id"] for entry in cp["attractors"]
                ]
                atoms_raw = [
                    atoms[int(entry["cell_id"]) - 1] for entry in cp["attractors"]
                ]
                # get argsort of atoms_raw
                argsort = np.argsort(bond_dict[cp["id"]]["atom_ids"])
                #print(bond_dict[cp["id"]]["atom_ids"], argsort)
                # sort atoms_raw
                atoms_raw = [atoms_raw[i] for i in argsort]
                # sort atom_ids
                bond_dict[cp["id"]]["atom_ids"] = [int(bond_dict[cp["id"]]["atom_ids"][i])-1 for i in argsort]
                
                bond_dict[cp["id"]]["atoms"] = atoms_raw
                bond_dict[cp["id"]]["distance"] = (
                    cp["attractors"][0]["distance"] * bohr_to_ang
                    + cp["attractors"][1]["distance"] * bohr_to_ang
                )

                if features["bond"] != []:
                    for feature in features["bond"]:
                        if feature in cp_features[cp_ind]:
                            if feature == "ellipticity":
                                bond_dict[cp["id"]]["extra_feat_bond_" + translation[feature]] = np.abs(cp_features[cp_ind]["hessian_eigenvalues"][0]) / np.abs(cp_features[cp_ind]["hessian_eigenvalues"][1]) - 1
                            elif feature == "hessian_eigenvalues":
                                bond_dict[cp["id"]]["extra_feat_bond_" + translation[feature]] = cp_features[cp_ind]["hessian_eigenvalues"][0]           
                            else:
                                bond_dict[cp["id"]]["extra_feat_bond_" + translation[feature]] = cp_features[cp_ind][feature]
                        

    bonds = []
    for cpid in bond_dict:
        # merge bonds with same atoms
        bonds.append([int(entry) for entry in bond_dict[cpid]["atom_ids"]])

    processed_dict = {}
    processed_dict["bonds"] = bonds # bonds
    processed_dict["bond_cps"] = bond_dict # bond info
    processed_dict["atom_cps"] = atom_dict # atom info
    return processed_dict

features = {
    "atom": ['field', 'gradient_norm', 'laplacian', 'hessian_eigenvalues', 'ellipticity'], 
    "bond": ['field', 'gradient_norm', 'laplacian', 'hessian_eigenvalues', 'ellipticity']
}
ret_dict = parse_critic2(cro, features=features)

In [103]:
ret_dict["bond_cps"]

{27: {'field': 0.35165770877580504,
  'atom_ids': [13, 15],
  'atoms': ['O', 'H'],
  'distance': 0.9634409624906461,
  'extra_feat_bond_field': 0.35165770877580504,
  'extra_feat_bond_grad_norm': 9.131307291379716e-15,
  'extra_feat_bond_lap_norm': -2.125022321690918,
  'extra_feat_bond_eig_hess': -1.8255798201026927},
 28: {'field': 0.34365824138759926,
  'atom_ids': [1, 2],
  'atoms': ['O', 'H'],
  'distance': 0.9693878706353847,
  'extra_feat_bond_field': 0.34365824138759926,
  'extra_feat_bond_grad_norm': 7.463259681867457e-15,
  'extra_feat_bond_lap_norm': -1.8462112184835628,
  'extra_feat_bond_eig_hess': -1.724552144742667},
 29: {'field': 0.31757654512343003,
  'atom_ids': [14, 16],
  'atoms': ['C', 'C'],
  'distance': 1.3924803828870886,
  'extra_feat_bond_field': 0.31757654512343003,
  'extra_feat_bond_grad_norm': 8.094069080035987e-13,
  'extra_feat_bond_lap_norm': -0.8969110966258571,
  'extra_feat_bond_eig_hess': -0.6774805365482282},
 30: {'field': 0.3168360974372714,
  '

In [104]:
ret_dict["bonds"]

[[13, 15],
 [1, 2],
 [14, 16],
 [0, 7],
 [14, 20],
 [0, 3],
 [18, 19],
 [4, 5],
 [6, 7],
 [16, 17],
 [17, 18],
 [5, 6],
 [3, 4],
 [19, 20],
 [0, 1],
 [17, 22],
 [19, 24],
 [18, 23],
 [16, 21],
 [6, 11],
 [4, 9],
 [5, 10],
 [7, 12],
 [3, 8],
 [20, 25],
 [13, 14],
 [2, 13],
 [8, 13],
 [1, 21]]